In [1]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None
import numpy as np
 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set('talk')
sns.set_style('white')

import lightgbm
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, RepeatedKFold
from sklearn.metrics import roc_auc_score, matthews_corrcoef

from hyperopt import Trials, STATUS_OK, tpe, fmin, hp
from timeit import default_timer as timer
import pickle
from numba import jit

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import glob

files = glob.glob('hyperopt/*')
for f in files:
    os.remove(f)
    
files = glob.glob('model/*')
for f in files:
    os.remove(f)

In [3]:
train = pd.read_csv('../../data/data-lgbm/train_data.csv')
test = pd.read_csv('../../data/data-lgbm/test_data.csv')
test_rowid = pd.read_csv('../../data/test.csv', usecols=['row_id'])

In [4]:
RANDOM_SEED = 42
FOLDS = 5
N_REPEATS = 1

In [5]:
X = train.drop(columns=['open_flag'])
columns_order = X.columns
X_test = test[columns_order].values

X = X.values
Y = train['open_flag'].values
print('Number of features: {}'.format(X.shape[1]))

Number of features: 23


In [6]:
def MCC(y_prob, dataset):
    y_true = dataset.get_label()
    mcc = eval_mcc(y_true, y_prob)
    return 'MCC', mcc, True

In [7]:
@jit
def mcc(tp, tn, fp, fn):
    sup = tp * tn - fp * fn
    inf = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    if inf==0:
        return 0
    else:
        return sup / np.sqrt(inf)
@jit    
def eval_mcc(y_true, y_prob, show=False):
    idx = np.argsort(y_prob)
    y_true_sort = y_true[idx]
    n = y_true.shape[0]
    nump = 1.0 * np.sum(y_true) # number of positive
    numn = n - nump # number of negative
    tp = nump
    tn = 0.0
    fp = numn
    fn = 0.0
    best_mcc = 0.0
    best_id = -1
    prev_proba = -1
    best_proba = -1
    mccs = np.zeros(n)
    for i in range(n):
        # all items with idx < i are predicted negative while others are predicted positive
        # only evaluate mcc when probability changes
        proba = y_prob[idx[i]]
        if proba != prev_proba:
            prev_proba = proba
            new_mcc = mcc(tp, tn, fp, fn)
            if new_mcc >= best_mcc:
                best_mcc = new_mcc
                best_id = i
                best_proba = proba
        mccs[i] = new_mcc
        if y_true_sort[i] == 1:
            tp -= 1.0
            fn += 1.0
        else:
            fp -= 1.0
            tn += 1.0
    if show:
        y_pred = (y_prob >= best_proba).astype(int)
        return best_proba, best_mcc, y_pred
    else:
        return best_mcc

In [8]:
skf = RepeatedKFold(n_splits=FOLDS, n_repeats=N_REPEATS, random_state=RANDOM_SEED)

In [9]:
categorical_features = ['country_code', 'grass_day', 'domain', 'attr_1', 'attr_2', 'attr_3']
categorical_index = [c for c, col in enumerate(columns_order) if col in categorical_features]

In [10]:
search_space = {
#     'device_type': hp.choice('device_type', ['cpu']),
#     'boosting': hp.choice('boosting', ['gbdt']),
'objective': hp.choice('objective', ['binary']),
'seed': hp.choice('seed', [RANDOM_SEED]),
#     'boost_from_average': hp.choice('boost_from_average', [False]), 
#     'bagging_freq': hp.choice('bagging_freq', [1]),
#     'use_missing': hp.choice('use_missing', [False]),
#     'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1)),
'min_gain_to_split': hp.uniform('min_gain_to_split', 0, 1),
#     'max_depth': hp.choice('max_depth', range(2, 6)),
'num_leaves': hp.choice('num_leaves', range(4, 33)),
#     'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
#     'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
'lambda_l1': hp.uniform('lambda_l1', 0, 10),
'lambda_l2': hp.uniform('lambda_l2', 0, 10),
#     'scale_pos_weight': hp.choice('scale_pos_weight', range(1, 8))
}
    
def opt(params):
    start = timer()
    oof_prob_arr = np.zeros(Y.shape)
    oof_prob_df = train[['open_flag']].copy()
    test_prob_arr = []
    test_prob_df = test_rowid.copy()
    fold_mcc = np.zeros(FOLDS*N_REPEATS)
    fold_threshold = np.zeros(FOLDS*N_REPEATS)
    for fold, (trn_ind, val_ind) in enumerate(skf.split(X)):
        X_train, X_valid = X[trn_ind], X[val_ind]
        Y_train, Y_valid = Y[trn_ind], Y[val_ind]
        evals_dic = {}
        train_data = lightgbm.Dataset(X_train, label=Y_train, categorical_feature=categorical_index, free_raw_data=True)
        valid_data = lightgbm.Dataset(X_valid, label=Y_valid, categorical_feature=categorical_index, reference=train_data, free_raw_data=True)
        gbm = lightgbm.train(params, train_data, valid_sets=valid_data, num_boost_round=5000, early_stopping_rounds=100, feval=MCC, evals_result=evals_dic, verbose_eval=False)
        
        train_prob = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
        threshold, mcc, _ = eval_mcc(Y_valid, train_prob, show=True)
        fold_mcc[fold], fold_threshold[fold] = mcc, threshold
        oof_prob_arr[val_ind] = train_prob
        test_prob = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        test_prob_arr.append(test_prob)
    
    oof_threshold, oof_mcc, _ = eval_mcc(Y, oof_prob_arr, show=True)
    mean_mcc, std_mcc = fold_mcc.mean(), fold_mcc.std()
    mean_threshold, std_threshold = fold_threshold.mean(), fold_threshold.std()
    loss = -mean_mcc
    run_time = timer() - start
    return {'loss': loss, 'params': params, 'train_time': run_time, 'status': STATUS_OK}

def run_trials():
    trials_step = 10  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 10  # initial max_trials. put something small to not have to wait

    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open('hyperopt/trials.hyperopt', 'rb'))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn=opt, space=search_space, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    # save the trials object
    with open('hyperopt/trials.hyperopt', 'wb') as f:
        pickle.dump(trials, f)

i = 0
while True:
    run_trials()
    i += 10
    if i == 50:
        print('Reached end of search')
        print('')
        break

bestParams = {}
trials = pickle.load(open('hyperopt/trials.hyperopt', 'rb'))
sorted_results = sorted(trials.results, key=lambda x: x['loss'])
results = {}
for dic in sorted_results:
    loss = dic['loss']
    results[loss] = dic['params']
    
best_loss = list(results.keys())[0]
bestParams['lgbm'] = results[best_loss]
print('Best loss: {:.4f}'.format(best_loss))
print('{}: {}'.format('lgbm', bestParams['lgbm']))
print('')
print('*************************************************************************************')

bestLGBMparams = bestParams['lgbm']
oof_prob_arr = np.zeros(Y.shape)
oof_prob_df = train[['open_flag']].copy()
test_prob_arr = []
test_prob_df = test_rowid.copy()
fold_mcc = np.zeros(FOLDS*N_REPEATS)
fold_threshold = np.zeros(FOLDS*N_REPEATS)

for fold, (trn_ind, val_ind) in enumerate(skf.split(X)):
    print('Training fold {}...'.format(fold))
    X_train, X_valid = X[trn_ind], X[val_ind]
    Y_train, Y_valid = Y[trn_ind], Y[val_ind]
    train_data = lightgbm.Dataset(X_train, label=Y_train, categorical_feature=categorical_index)
    valid_data = lightgbm.Dataset(X_valid, label=Y_valid, categorical_feature=categorical_index, reference=train_data)
    model = lightgbm.train(bestLGBMparams, train_data, valid_sets=valid_data, num_boost_round=5000, feval=MCC, early_stopping_rounds=100, verbose_eval=False)
    
    train_prob = model.predict(X_valid, num_iteration=model.best_iteration)
    threshold, mcc, _ = eval_mcc(Y_valid, train_prob, show=True)
    fold_mcc[fold], fold_threshold[fold] = mcc, threshold
    oof_prob_arr[val_ind] = train_prob
    test_prob = model.predict(X_test, num_iteration=model.best_iteration)
    test_prob_arr.append(test_prob)

oof_threshold, oof_mcc, _ = eval_mcc(Y, oof_prob_arr, show=True)
mean_mcc, std_mcc = fold_mcc.mean(), fold_mcc.std()
mean_threshold, std_threshold = fold_threshold.mean(), fold_threshold.std()
print('')
print('Fold Average MCC: {:.4f} \u00B1 {:.4f}'.format(mean_mcc, std_mcc))
print('Fold Average threshold: {:.4f} \u00B1 {:.4f}'.format(mean_threshold, std_threshold))
print('')
print('Out of Fold MCC: {:.4f}'.format(oof_mcc))
print('Out of Fold threshold: {:.4f}'.format(oof_threshold))

def bestThreshold(prob, minimum, maximum):
    true_positive = 2476
    diff = np.inf
    best_threshold = -1
    for thr in np.arange(minimum, maximum, 0.001):
        positive = (prob > thr).sum()*0.3
        current_diff = np.abs(true_positive - positive)
        if current_diff < diff:
            diff = current_diff
            best_threshold = thr
    return best_threshold

oof_prob_df['prob'] = oof_prob_arr
test_prob_df['prob'] = np.mean(test_prob_arr, axis=0).flatten()

minimum = float('{:.2f}'.format(mean_threshold-1.5*std_threshold))
maximum = float('{:.2f}'.format(mean_threshold+1.5*std_threshold))
best_threshold = bestThreshold(test_prob_df['prob'].values, minimum, maximum)
print('')
print('Threshold: {:.4f}'.format(best_threshold))

test_prob_df['open_flag'] = (test_prob_df['prob'] >= best_threshold).astype(int)
oof_prob_df.to_csv('train/prob.csv', index=False)
test_prob_df[['row_id', 'prob']].to_csv('test/prob.csv', index=False)
test_prob_df[['row_id', 'open_flag']].to_csv('pred-v2-2.csv', index=False)

100%|███████████████████████████████████████████████| 10/10 [00:42<00:00,  4.29s/trial, best loss: -0.5396515954918997]
Found saved Trials! Loading...
Rerunning from 10 trials to 20 (+10) trials
100%|███████████████████████████████████████████████| 20/20 [00:46<00:00,  2.33s/trial, best loss: -0.5396515954918997]
Found saved Trials! Loading...
Rerunning from 20 trials to 30 (+10) trials
100%|███████████████████████████████████████████████| 30/30 [00:50<00:00,  1.68s/trial, best loss: -0.5408003470817686]
Found saved Trials! Loading...
Rerunning from 30 trials to 40 (+10) trials
100%|███████████████████████████████████████████████| 40/40 [00:41<00:00,  1.05s/trial, best loss: -0.5408003470817686]
Found saved Trials! Loading...
Rerunning from 40 trials to 50 (+10) trials
100%|███████████████████████████████████████████████| 50/50 [00:41<00:00,  1.20trial/s, best loss: -0.5419921073682545]
Reached end of search

Best loss: -0.5420
lgbm: {'lambda_l1': 3.2438460371091287, 'lambda_l2': 2.093